In [130]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [131]:
transactions = pd.read_csv('Transactions.csv')
products = pd.read_csv('Products.csv')
customers = pd.read_csv('Customers.csv')

In [132]:
#I am not checking for null types or missing values and such as i checked for all those things in EDA part of Assignments
dataset = transactions.merge(products,on = "ProductID",how = 'left').merge(customers,on = "CustomerID",how = 'left')
dataset['TransactionDate'] = pd.to_datetime(dataset['TransactionDate'])
dataset['SignupDate'] = pd.to_datetime(dataset['SignupDate'])
dataset = dataset.drop(columns='Price_y',axis=1)
dataset.rename(columns={'Price_x':'Price'},inplace=True)
print(dataset)

    TransactionID CustomerID ProductID     TransactionDate  Quantity  \
0          T00001      C0199      P067 2024-08-25 12:38:23         1   
1          T00112      C0146      P067 2024-05-27 22:23:54         1   
2          T00166      C0127      P067 2024-04-25 07:38:55         1   
3          T00272      C0087      P067 2024-03-26 22:55:37         2   
4          T00363      C0070      P067 2024-03-21 15:10:10         3   
..            ...        ...       ...                 ...       ...   
995        T00496      C0118      P037 2024-10-24 08:30:27         1   
996        T00759      C0059      P037 2024-06-04 02:15:24         3   
997        T00922      C0018      P037 2024-04-05 13:05:32         4   
998        T00959      C0115      P037 2024-09-29 10:16:02         2   
999        T00992      C0024      P037 2024-04-21 10:52:24         1   

     TotalValue   Price                      ProductName     Category  \
0        300.68  300.68  ComfortLiving Bluetooth Speaker  Elec

In [133]:
dataset.describe()

,TransactionDate,Quantity,TotalValue,Price,SignupDate
count,1000,1000.000000,1000.000000,1000.00000,1000
mean,2024-06-23 15:33:02.768999936,2.537000,689.995560,272.55407,2023-07-09 02:49:55.199999744
min,2023-12-30 15:29:12,1.000000,16.080000,16.08000,2022-01-22 00:00:00
25%,2024-03-25 22:05:34.500000,2.000000,295.295000,147.95000,2022-09-17 12:00:00
50%,2024-06-26 17:21:52.500000,3.000000,588.880000,299.93000,2023-07-23 00:00:00
75%,2024-09-19 14:19:57,4.000000,1011.660000,404.40000,2024-04-12 00:00:00
max,2024-12-28 11:00:00,4.000000,1991.040000,497.76000,2024-12-28 00:00:00
std,NaN,1.117981,493.144478,140.73639,NaN


In [134]:
regions_encoded = pd.get_dummies(dataset[['CustomerID','Region']], columns=['Region'])
print(regions_encoded.columns)

Index(['CustomerID', 'Region_Asia', 'Region_Europe', 'Region_North America',
       'Region_South America'],
      dtype='object')


In [135]:
customer_aggregates = dataset.groupby('CustomerID').agg(
    TotalTransactions=('TransactionID', 'count'),
    TotalQuantity=('Quantity', 'sum'),
    TotalValue=('TotalValue', 'sum'),
    AvgSpendingPerTransaction = ('TotalValue', lambda x: x.sum() / x.count())
).reset_index()
print(customer_aggregates)

    CustomerID  TotalTransactions  TotalQuantity  TotalValue  \
0        C0001                  5             12     3354.52   
1        C0002                  4             10     1862.74   
2        C0003                  4             14     2725.38   
3        C0004                  8             23     5354.88   
4        C0005                  3              7     2034.24   
..         ...                ...            ...         ...   
194      C0196                  4             12     4982.88   
195      C0197                  3              9     1928.65   
196      C0198                  2              3      931.83   
197      C0199                  4              9     1979.28   
198      C0200                  5             16     4758.60   

     AvgSpendingPerTransaction  
0                   670.904000  
1                   465.685000  
2                   681.345000  
3                   669.360000  
4                   678.080000  
..                         ...  


In [136]:
category_pivot = dataset.pivot_table(
    index='CustomerID',
    columns='Category',
     values='Quantity',
    aggfunc='sum',
    fill_value=0
).reset_index()
print(category_pivot)


Category CustomerID  Books  Clothing  Electronics  Home Decor
0             C0001      2         0            7           3
1             C0002      0         4            0           6
2             C0003      0         4            4           6
3             C0004      8         0            6           9
4             C0005      0         0            4           3
..              ...    ...       ...          ...         ...
194           C0196      3         4            0           5
195           C0197      0         0            6           3
196           C0198      0         2            1           0
197           C0199      0         0            3           6
198           C0200      4         7            1           4

[199 rows x 5 columns]


In [137]:
customer_features = customer_aggregates.merge(regions_encoded,on='CustomerID',how='left').merge(category_pivot,on='CustomerID',how='left')
customer_features = customer_features.drop_duplicates(subset='CustomerID').reset_index()
customer_features = customer_features.drop(columns='index',axis=1)
print(customer_features)

    CustomerID  TotalTransactions  TotalQuantity  TotalValue  \
0        C0001                  5             12     3354.52   
1        C0002                  4             10     1862.74   
2        C0003                  4             14     2725.38   
3        C0004                  8             23     5354.88   
4        C0005                  3              7     2034.24   
..         ...                ...            ...         ...   
194      C0196                  4             12     4982.88   
195      C0197                  3              9     1928.65   
196      C0198                  2              3      931.83   
197      C0199                  4              9     1979.28   
198      C0200                  5             16     4758.60   

     AvgSpendingPerTransaction  Region_Asia  Region_Europe  \
0                   670.904000        False          False   
1                   465.685000         True          False   
2                   681.345000        False  

In [138]:
import datetime
last_purchase_customers = dataset.groupby('CustomerID')['TransactionDate'].max().reset_index()
days_since_last_purchase = datetime.datetime.now() - last_purchase_customers['TransactionDate']
customer_features['DaysSinceLastPurchase'] = days_since_last_purchase.dt.days
print(customer_features['DaysSinceLastPurchase'])

0       82
1       52
2      152
3       32
4       81
      ... 
194     40
195     27
196    111
197     90
198     44
Name: DaysSinceLastPurchase, Length: 199, dtype: int64


In [139]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
customer_features['TotalValue'] = scaler.fit_transform(customer_features[['TotalValue']])
customer_features['DaysSinceLastPurchase'] = scaler.fit_transform(customer_features[['DaysSinceLastPurchase']])
customer_features['TotalTransactions'] = scaler.fit_transform(customer_features[['TotalTransactions']])
customer_features['TotalQuantity'] = scaler.fit_transform(customer_features[['TotalQuantity']])
customer_features['AvgSpendingPerTransaction'] = scaler.fit_transform(customer_features[['AvgSpendingPerTransaction']])

print(customer_features)

    CustomerID  TotalTransactions  TotalQuantity  TotalValue  \
0        C0001                0.4       0.354839    0.308942   
1        C0002                0.3       0.290323    0.168095   
2        C0003                0.3       0.419355    0.249541   
3        C0004                0.7       0.709677    0.497806   
4        C0005                0.2       0.193548    0.184287   
..         ...                ...            ...         ...   
194      C0196                0.3       0.354839    0.462684   
195      C0197                0.2       0.258065    0.174318   
196      C0198                0.1       0.064516    0.080203   
197      C0199                0.3       0.258065    0.179098   
198      C0200                0.4       0.483871    0.441508   

     AvgSpendingPerTransaction  Region_Asia  Region_Europe  \
0                     0.474336        False          False   
1                     0.308940         True          False   
2                     0.482751        False  

In [140]:
from sklearn.metrics.pairwise import cosine_similarity
similarity_matrix = cosine_similarity(customer_features.drop(columns='CustomerID'))
print(similarity_matrix)

[[1.         0.3164972  0.71527379 ... 0.36689163 0.72661002 0.38017404]
 [0.3164972  1.         0.86024079 ... 0.45070416 0.73066705 0.79996009]
 [0.71527379 0.86024079 1.         ... 0.59352677 0.85296441 0.74189584]
 ...
 [0.36689163 0.45070416 0.59352677 ... 1.         0.24867467 0.67320372]
 [0.72661002 0.73066705 0.85296441 ... 0.24867467 1.         0.44164937]
 [0.38017404 0.79996009 0.74189584 ... 0.67320372 0.44164937 1.        ]]


In [141]:
customer_similarity_df = pd.DataFrame(
    similarity_matrix,
    index=customer_features['CustomerID'],
    columns=customer_features.index
)
print(customer_similarity_df)

                 0         1         2         3         4         5    \
CustomerID                                                               
C0001       1.000000  0.316497  0.715274  0.804402  0.914503  0.361022   
C0002       0.316497  1.000000  0.860241  0.552602  0.514999  0.655393   
C0003       0.715274  0.860241  1.000000  0.706350  0.804239  0.594429   
C0004       0.804402  0.552602  0.706350  1.000000  0.741906  0.706496   
C0005       0.914503  0.514999  0.804239  0.741906  1.000000  0.259506   
...              ...       ...       ...       ...       ...       ...   
C0196       0.378539  0.881803  0.775484  0.715931  0.421942  0.903547   
C0197       0.947296  0.368284  0.746670  0.690015  0.953806  0.194975   
C0198       0.366892  0.450704  0.593527  0.187395  0.331694  0.467662   
C0199       0.726610  0.730667  0.852964  0.787917  0.867792  0.377846   
C0200       0.380174  0.799960  0.741896  0.605266  0.373779  0.922571   

                 6         7         

In [142]:
top_3_similar_customers = customer_similarity_df.apply(
    lambda row: list(zip(row.sort_values(ascending=False).iloc[1:4].index.tolist(),
                         [round(score, 3) for score in row.sort_values(ascending=False).iloc[1:4].tolist()])),
    axis=1
)
recommendations = pd.DataFrame({
    'CustomerID': top_3_similar_customers.index,
    'Top_3_Similar_Customers_with_Similarity_Score': top_3_similar_customers
})
recommendations
recommendations.head(n=20)

,CustomerID,Top_3_Similar_Customers_with_Similarity_Score
CustomerID,,
C0001,C0001,"[(156, 0.974), (50, 0.964), (44, 0.956)]"
C0002,C0002,"[(158, 0.998), (177, 0.984), (132, 0.977)]"
C0003,C0003,"[(169, 0.978), (193, 0.972), (99, 0.97)]"
C0004,C0004,"[(145, 0.972), (147, 0.965), (11, 0.964)]"
C0005,C0005,"[(6, 0.99), (195, 0.954), (126, 0.951)]"
C0006,C0006,"[(81, 0.977), (23, 0.968), (134, 0.967)]"
C0007,C0007,"[(4, 0.99), (162, 0.968), (126, 0.964)]"
C0008,C0008,"[(148, 0.972), (30, 0.962), (92, 0.958)]"
C0009,C0009,"[(196, 0.998), (91, 0.901), (110, 0.895)]"


In [143]:
recommendations.head(n=20).to_csv('Lookalike.csv',index=False)


In [144]:
print(customer_features.columns)

Index(['CustomerID', 'TotalTransactions', 'TotalQuantity', 'TotalValue',
       'AvgSpendingPerTransaction', 'Region_Asia', 'Region_Europe',
       'Region_North America', 'Region_South America', 'Books', 'Clothing',
       'Electronics', 'Home Decor', 'DaysSinceLastPurchase'],
      dtype='object')
